<a href="https://colab.research.google.com/github/ArunK-ML/Project---TripFare-Predicting-Urban-Taxi-Fare-with-Machine-Learning/blob/main/Two_Model_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.3 MB/s eta 0:00:00


In [1]:
!pip install streamlit streamlit_option_menu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.3/829.3 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 95.3 MB/s eta 0:00:00


In [14]:
%%writefile TripFare_streamlit_app_finall.py
import streamlit as st
import pickle
import numpy as np
import pandas as pd

# -------------------------------
# Load Models & Scaler
# -------------------------------
@st.cache_resource
def load_artifacts():
    with open("FareAmount_prediction_model.pkl", "rb") as f1:
        fare_model = pickle.load(f1)
    with open("TotalAmount_prediction_model.pkl", "rb") as f2:
        total_model = pickle.load(f2)
    with open("scaler.pkl", "rb") as f3:
        scaler = pickle.load(f3)
    return fare_model, total_model, scaler

fare_model, total_model, scaler = load_artifacts()

# -------------------------------
# Streamlit App
# -------------------------------
st.title("Taxi Fare & Total Amount Prediction 🚖")

# =====================================================
# Step 1: Predict Fare Amount
# =====================================================
st.header("Step 1: Predict Fare Amount")

duration = st.number_input("Trip Duration (minutes)", min_value=1.0, value=10.0)
trip_distance = st.number_input("Trip Distance (miles)", min_value=0.1, value=2.5)
passenger_count = st.number_input("Passenger Count", min_value=1, max_value=3, value=1)
store_and_fwd_flag = st.selectbox("Store and Forward Flag", [0, 1], index=0)  # 0 = No, 1 = Yes
payment_type = st.selectbox("Payment Type", [0, 1], index=0)  # 0 = Cash, 1 = Card

fare_pred = None
if st.button("Predict Fare Amount"):
    # compute fare_per_mile feature (placeholder if not used properly)
    fare_per_mile = 0.0

    # Create DataFrame in same column order as training
    input_data = pd.DataFrame([[passenger_count, store_and_fwd_flag, payment_type,
                                trip_distance, duration, fare_per_mile]],
                                columns=['passenger_count', 'store_and_fwd_flag', 'payment_type',
                                         'trip_distance_miles', 'Duration', 'fare_per_mile'])

    # Scale selected features
    input_data[['trip_distance_miles', 'Duration', 'fare_per_mile']] = scaler.transform(
        input_data[['trip_distance_miles', 'Duration', 'fare_per_mile']]
    )

    # Predict fare
    fare_pred = fare_model.predict(input_data)[0]
    st.success(f"Predicted Fare Amount: ${fare_pred:.2f}")

# =====================================================
# Step 2: Predict Total Amount
# =====================================================
st.header("Step 2: Predict Total Amount")

use_total = st.radio("Do you want to predict Total Amount?", ["No", "Yes"])

if use_total == "Yes":
    st.subheader("Enter charges for Total Amount prediction")

    # If fare predicted, show it as default
    if fare_pred is not None:
        fare_amount = st.number_input("Fare Amount", value=float(fare_pred))
    else:
        fare_amount = st.number_input("Fare Amount", value=0.0)

    extra = st.number_input("Extra Charges", value=0.0)
    mta_tax = st.number_input("MTA Tax", value=0.5)
    tip_amount = st.number_input("Tip Amount", value=0.0)
    tolls_amount = st.number_input("Tolls Amount", value=0.0)
    improvement_surcharge = st.number_input("Improvement Surcharge", value=0.3)

    if st.button("Predict Total Amount"):
        input_data2 = pd.DataFrame([[fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge]],
                                   columns=['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge'])

        total_pred = total_model.predict(input_data2)[0]
        st.success(f"Predicted Total Amount: ${total_pred:.2f}")


Overwriting TripFare_streamlit_app_finall.py


In [4]:
!wget -q -O - ipv4.icanhazip.com

34.91.131.80


In [5]:
!pip install pyngrok

In [15]:
!streamlit run TripFare_streamlit_app_finall.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.91.131.80:8501

⠴⠦⠧⠇⠏⠋your url is: https://all-jars-invent.loca.lt
  Stopping...
^C
